In [ ]:
pip install --ignore-installed torchvision --user

In [ ]:
# !pip uninstall torch 
# !pip install torch 

In [36]:
pip install pycocotools

  Obtaining dependency information for pycocotools from https://files.pythonhosted.org/packages/24/b2/ef28a34cf6ca50b6b2f7ad81e5837ed45c252ffef22f5a704b94141ea842/pycocotools-2.0.7-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/85.8 kB ? eta -:--:--
   ------------------- -------------------- 41.0/85.8 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 85.8/85.8 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install Cython

In [ ]:
!pip install torch==2.3.1 torchvision==0.18.1

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
import os
import numpy as np
import cv2
from torch.utils.data import DataLoader
import xml.etree.ElementTree as ET
import torchvision.transforms as T

from keras.preprocessing.image import img_to_array
import os
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# Train model

In [34]:
# Paths to images and annotations
image_dir = r"C:\Users\User\Downloads\dataset22\dataset2\images"
annotation_dir = r"C:\Users\User\Downloads\dataset22\dataset2\annotations"

# Collect all image and annotation file names
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])
annotation_files = sorted([f for f in os.listdir(annotation_dir) if f.endswith('.xml')])

# Check if the lists are correctly populated
print(f"Number of images: {len(image_files)}")
print(f"Number of annotations: {len(annotation_files)}")

# Check if the number of image files matches the number of annotation files
assert len(image_files) == len(annotation_files), "Number of images and annotations do not match"

# Split the dataset into training and testing sets
train_images, test_images, train_annotations, test_annotations = train_test_split(
    image_files, annotation_files, train_size=100, test_size=28, random_state=42)

# Define the label mapping
label_mapping = {
    'obj1': 'Water Bottle',
    'obj2': 'Milk Bottle',
    'obj3': 'Tetra Pack',
    'obj4': 'Can'
}

# Update label_map to use the actual object names
label_map = {'Water Bottle': 1, 'Milk Bottle': 2, 'Tetra Pack': 3, 'Can': 4}

def parse_annotation(annotation_file):
    tree = ET.parse(annotation_file)
    root = tree.getroot()
    
    objects = []
    for obj in root.findall('object'):
        obj_name = obj.find('name').text
        if obj_name in label_mapping:
            obj_name = label_mapping[obj_name]
        bbox = obj.find('bndbox')
        bbox = [int(bbox.find('xmin').text), int(bbox.find('ymin').text),
                int(bbox.find('xmax').text), int(bbox.find('ymax').text)]
        objects.append((obj_name, bbox))
    
    return objects

class CustomDataset(Dataset):
    def __init__(self, image_files, annotation_files, image_dir, annotation_dir, transform=None):
        self.image_files = image_files
        self.annotation_files = annotation_files
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.label_map = label_map
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        ann_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        
        img = Image.open(img_path).convert("RGB")
        annotations = parse_annotation(ann_path)
        
        boxes = []
        labels = []
        for ann in annotations:
            if ann[0] not in self.label_map:
                print(f"Unexpected label found: {ann[0]}")
                continue  # Skip unexpected labels
            boxes.append(ann[1])
            labels.append(self.label_map[ann[0]])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, target

# Create datasets
train_dataset = CustomDataset(train_images, train_annotations, image_dir, annotation_dir, transform=transforms.ToTensor())
test_dataset = CustomDataset(test_images, test_annotations, image_dir, annotation_dir, transform=transforms.ToTensor())

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Define the model
import torchvision.models.detection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model(num_classes):
    # Load a pre-trained model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

# Number of classes (4 objects + 1 background)
num_classes = 5

# Initialize the model
model = get_model(num_classes)

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# Define the learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train the model
num_epochs = 10

# Check if CUDA is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for images, targets in train_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)

        # Calculate the total loss
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    
    # Update the learning rate
    lr_scheduler.step()

    print(f"Epoch {epoch+1} finished with loss {losses.item()}")


Number of images: 128
Number of annotations: 128
Epoch 1 finished with loss 0.30721786618232727
Epoch 2 finished with loss 0.16368253529071808
Epoch 3 finished with loss 0.11984279751777649
Epoch 4 finished with loss 0.07419842481613159
Epoch 5 finished with loss 0.029352128505706787
Epoch 6 finished with loss 0.057953108102083206
Epoch 7 finished with loss 0.13673891127109528
Epoch 8 finished with loss 0.05913308635354042
Epoch 9 finished with loss 0.07079485803842545
Epoch 10 finished with loss 0.0630192831158638


# Evaluate model

In [39]:
import torch
from sklearn.metrics import precision_recall_fscore_support

def evaluate_model(model, data_loader, device, iou_threshold=0.5):
    model.eval()
    true_positive = 0
    false_positive = 0
    false_negative = 0

    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)
            
            for target, output in zip(targets, outputs):
                true_boxes = target['boxes'].cpu()
                true_labels = target['labels'].cpu()
                pred_boxes = output['boxes'].cpu()
                pred_labels = output['labels'].cpu()

                matched_indices = []

                for i, true_box in enumerate(true_boxes):
                    if true_labels[i] not in pred_labels:
                        false_negative += 1
                        continue

                    ious = torchvision.ops.box_iou(true_box.unsqueeze(0), pred_boxes).squeeze(0)
                    best_iou_idx = torch.argmax(ious)
                    if ious[best_iou_idx] >= iou_threshold:
                        if best_iou_idx not in matched_indices:
                            matched_indices.append(best_iou_idx)
                            true_positive += 1
                        else:
                            false_negative += 1
                    else:
                        false_negative += 1

                false_positive += len(pred_boxes) - len(matched_indices)

    precision = true_positive / (true_positive + false_positive) if true_positive + false_positive > 0 else 0
    recall = true_positive / (true_positive + false_negative) if true_positive + false_negative > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1

# Load the model to the appropriate device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Create the test data loader
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Evaluate the model
precision, recall, f1 = evaluate_model(model, test_loader, device)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Precision: 0.3008
Recall: 1.0000
F1 Score: 0.4625


# class wise precision recall

In [40]:
import torch
import torchvision
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict

def evaluate_model_classwise(model, data_loader, device, iou_threshold=0.5):
    model.eval()
    
    class_metrics = defaultdict(lambda: {'true_positive': 0, 'false_positive': 0, 'false_negative': 0})

    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)

            for target, output in zip(targets, outputs):
                true_boxes = target['boxes'].cpu()
                true_labels = target['labels'].cpu()
                pred_boxes = output['boxes'].cpu()
                pred_labels = output['labels'].cpu()

                matched_indices = set()

                for i, true_box in enumerate(true_boxes):
                    true_label = true_labels[i].item()
                    if true_label not in pred_labels:
                        class_metrics[true_label]['false_negative'] += 1
                        continue

                    ious = torchvision.ops.box_iou(true_box.unsqueeze(0), pred_boxes).squeeze(0)
                    best_iou_idx = torch.argmax(ious)
                    if ious[best_iou_idx] >= iou_threshold:
                        if best_iou_idx not in matched_indices:
                            matched_indices.add(best_iou_idx)
                            class_metrics[true_label]['true_positive'] += 1
                        else:
                            class_metrics[true_label]['false_negative'] += 1
                    else:
                        class_metrics[true_label]['false_negative'] += 1

                for i, pred_label in enumerate(pred_labels):
                    if i not in matched_indices:
                        class_metrics[pred_label.item()]['false_positive'] += 1

    precisions = {}
    recalls = {}
    f1_scores = {}

    for cls, metrics in class_metrics.items():
        tp = metrics['true_positive']
        fp = metrics['false_positive']
        fn = metrics['false_negative']
        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
        precisions[cls] = precision
        recalls[cls] = recall
        f1_scores[cls] = f1

    return precisions, recalls, f1_scores

# Load the model to the appropriate device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Create the test data loader
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Evaluate the model
precisions, recalls, f1_scores = evaluate_model_classwise(model, test_loader, device)

for cls in precisions.keys():
    print(f"Class {cls}:")
    print(f"  Precision: {precisions[cls]:.4f}")
    print(f"  Recall: {recalls[cls]:.4f}")
    print(f"  F1 Score: {f1_scores[cls]:.4f}")


Class 4:
  Precision: 0.1789
  Recall: 1.0000
  F1 Score: 0.3036
Class 3:
  Precision: 0.2289
  Recall: 1.0000
  F1 Score: 0.3725
Class 1:
  Precision: 0.2899
  Recall: 1.0000
  F1 Score: 0.4494
Class 2:
  Precision: 0.2466
  Recall: 1.0000
  F1 Score: 0.3956
